In [52]:
import os
import pandas as pd
import re
from utils import *

import nltk
from nltk.tokenize import TweetTokenizer
import gensim
import pyLDAvis.gensim

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
parent_path = '/Users/chenjianyu/Library/Mobile Documents/com~apple~CloudDocs/SMU/SMU Module Materials/Y2S2/SMT203 Computational Social Sci/Covid-19-Singapore-Analysis'

In [3]:
social_media_data_folder_paths = [
    f'{parent_path}/Data/Hardwarezone Data/Cleaned Data', 
    f'{parent_path}/Data/Twitter Data/Cleaned Data/Policies/Combined',
    f'{parent_path}/Data/Facebook Data/Cleaned Data/Policies/Combined',
    f'{parent_path}/Data/Instagram Data/Cleaned Data/Policies/Combined',
    f'{parent_path}/Data/Reddit Data/Cleaned Data/Policies/Combined',
]

In [13]:
stop_list = nltk.corpus.stopwords.words('english')

In [45]:
def get_policy_data(policy, folders):
    frames = []
    for folder in folders:
        files = [file for file in os.listdir(folder) if file.endswith('.csv')]
        for file in files:
            if policy.lower() in file.lower():
                df = pd.read_csv(f'{folder}/{file}')
                df = df[['Comments', 'Comment Datetime', 'actionable', 'valuable']]
                frames.append(df)
                break
    final_df = pd.concat(frames, ignore_index=True)
    return final_df

def corpus2docs(df):
    docs1 = [TweetTokenizer().tokenize(comment) for comment in df['Comments']]
    docs2 = [[w.lower() for w in doc] for doc in docs1]
    docs3 = [[w for w in doc if re.search('^[a-z]+$', w)] for doc in docs2]
    docs4 = [[w for w in doc if w not in stop_list] for doc in docs3]
    return docs4

def docs2vecs(docs, dic):
    vecs = [dic.doc2bow(doc) for doc in docs]
    return vecs

In [9]:
df = get_policy_data('stay home notice', social_media_data_folder_paths)

In [10]:
valuable_comments = get_valuable_comments(df)
actionable_comments = get_actionable_comments(df)

In [12]:
print(len(valuable_comments))
print(len(actionable_comments))
print(len(df))

366
105
1582


In [40]:
valuable_docs = corpus2docs(valuable_comments)

In [44]:
valuable_dic = gensim.corpora.Dictionary(valuable_docs)

In [46]:
valuable_vecs = docs2vecs(valuable_docs, valuable_dic)

In [54]:
lda_shn = gensim.models.ldamodel.LdaModel(corpus=valuable_vecs, id2word=valuable_dic, num_topics=7)

/Users/chenjianyu/Library/Python/3.9/lib/python/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [56]:
topics = lda_shn.show_topics(7, 20)

for i in range(0, 7):
    print(topics[i])

(0, '0.008*"shn" + 0.006*"also" + 0.006*"covid" + 0.005*"get" + 0.005*"singapore" + 0.005*"need" + 0.005*"back" + 0.004*"country" + 0.004*"day" + 0.004*"hotel" + 0.004*"like" + 0.004*"days" + 0.004*"said" + 0.003*"people" + 0.003*"case" + 0.003*"testing" + 0.003*"mr" + 0.003*"cases" + 0.003*"tests" + 0.003*"still"')
(1, '0.014*"shn" + 0.010*"test" + 0.010*"case" + 0.009*"days" + 0.007*"covid" + 0.007*"singapore" + 0.007*"back" + 0.007*"positive" + 0.006*"infection" + 0.006*"cases" + 0.006*"tested" + 0.006*"also" + 0.006*"still" + 0.006*"community" + 0.005*"january" + 0.005*"people" + 0.004*"like" + 0.004*"quarantine" + 0.004*"result" + 0.004*"one"')
(2, '0.011*"covid" + 0.009*"singapore" + 0.008*"people" + 0.007*"cases" + 0.007*"virus" + 0.006*"case" + 0.006*"shn" + 0.006*"community" + 0.005*"also" + 0.005*"infection" + 0.005*"one" + 0.005*"infected" + 0.005*"test" + 0.004*"positive" + 0.004*"day" + 0.004*"mask" + 0.004*"work" + 0.004*"would" + 0.004*"back" + 0.004*"leave"')
(3, '0.011

In [57]:
pyLDAvis.enable_notebook()
visual= pyLDAvis.gensim.prepare(lda_shn, valuable_vecs, valuable_dic)
pyLDAvis.save_html(visual, "topic_viz.html")

/Users/chenjianyu/Library/Python/3.9/lib/python/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
